In [1]:
print('begin')
scale_cadence = 3

import pandas as pd
import numpy  as np

from pygeodyn.pygeodyn_plot_scalingfactors import *
import os


begin


In [2]:
def plot_cleanformat_axes(fig, font_dict):
    rownum, colnum = fig._get_subplot_rows_columns()

    for i in rownum:
        if len(rownum)==1:
            L_ticklabel = True
        else:
            if i < len(rownum):
                L_ticklabel = False
            else:
                L_ticklabel = True
        fig.update_xaxes(### LINE at axis border
                          showline=True,
                          showticklabels=L_ticklabel,
    #                       tickformat= '%m/%d',
                          linecolor='black',
                          linewidth=1,
                         ### Major ticks
                          ticks='inside',
                          tickfont=font_dict,
                          mirror=True,
    #                       tickwidth=2,
    #                       ticklen=9,
                          tickcolor='grey',
    #                       tick0="2018-11-9" ,
    #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                          #### Minor Ticks
                           minor=dict(
                             dtick=86400000.0, # milliseconds in a day
                             tickwidth=1,
                             ticklen=4,
                             tickcolor='grey',
                             ticks='inside'),
                          ### GRID
                           gridcolor='gainsboro',
                           gridwidth=1,
                           layer='above traces',
                           tickangle=0,
                           row=i, col=1)
        fig.update_yaxes(showline=True,      # add line at x=0
                             showticklabels=True,
                             linecolor='black',  # line color
                             linewidth=1,        # line size
                         ticks='inside',     # ticks outside axis
                         tickfont=font_dict, # tick label font
                         mirror='allticks',  # add ticks to top/right axes
                         tickwidth=1,      # tick width
                         tickcolor='black',  # tick color
                         gridcolor='gainsboro',
                         gridwidth=1,
                         layer='above traces',
                         row=i, col=1)
    return(fig)

from pygeodyn.pygeodyn_plot_scalingfactors import *

coldict = {}
coldict['msis2']     = "#2ca02c"  # 'tab:green'
coldict['dtm2020_o'] = "#d62728"  # 'tab:red'
coldict['jb2008']    = "orange"   
coldict['hasdm_oc']  = "#1f77b4"     


## GFO Acc

In [3]:
month_list = ['jun']#,
gfo_acc_file = 'gfo_acc_june.csv'
ice_pod_file = 'icesat2_june.csv'
sp104_pod_file = 'spire104_june.csv'
sp113_pod_file = 'spire113_june.csv'
sp106_pod_file = 'spire106_june.csv'


In [4]:
if os.path.exists(gfo_acc_file) :
    print(gfo_acc_file, ' exists' )

elif os.path.exists(gfo_acc_file+'.bz2') :
    
    print('must bunzip')
    os.system('bunzip2 -v '+ gfo_acc_file+'.bz2')
# gfo_6month.csv.bz2
else:
    gfo_bigdf = {}

    for imonth,month in enumerate(month_list):
        if month=='jun':
            m_num = 6
            y_num = 2022

        path_gfo     = "/data/SatDragModelValidation/data/inputs/raw_inputdata/data_GRACEFO_accelerometer/"
        filename_gfo = f'GC_DNS_ACC_{y_num}_{m_num:02d}_v02c.txt'

        datapath_gfo     = path_gfo + filename_gfo

        headers = [
            'date',        #         Date (yyyy-mm-dd)
            'time',        #         Time (hh:mm:ss.sss)
            'time_system', #         Time system: UTC or GPS (differs per mission)
            'alt',         #  f10.3  Altitude (m), GRS80
            'lon',         #   f8.3  Geodetic longitude (deg), GRS80
            'lat',         #   f7.3  Geodetic latitude (deg), GRS80
            'lst',         #   f6.3  Local solar time (h)
            'arglat',      #   f7.3  Argument of latitude (deg)
            'dens_x',      #  e15.8  Density derived from accelerometer measurements (kg/m3)
            'dens_mean',   #  e15.8  Running orbit average of density (kg/m3)
            'flag_den',    #   f4.1  Flag for density: 0 = nominal data, 1 = anomalous data (-)
            'flag_orbitavg',#   f4.1  Flag for running orbit average density: 0 = nominal data, 1 = anomalous data (-)
                    ]

        print(month)
        gfo_bigdf[month] = pd.read_csv(datapath_gfo, 
                skiprows = 38, 
                sep = '\s+',
                names = headers,
                           )

        #Convert date from GPS to UTC
        date = pd.to_datetime(gfo_bigdf[month]['date']  \
                            + gfo_bigdf[month]['time'], \
                                    format='%Y-%m-%d%H:%M:%S.000') - pd.to_timedelta(18,'s')

        gfo_bigdf[month].insert(0, 'Date', date)

        del gfo_bigdf[month]['date'], gfo_bigdf[month]['time'], date
        del gfo_bigdf[month]['time_system']
        del gfo_bigdf[month]['dens_mean']
        del gfo_bigdf[month]['flag_den']
        del gfo_bigdf[month]['flag_orbitavg']


    #     resid_meas_summry = pd.concat([ gfo_bigdf, resid_meas_summry_iter])

    gfo_acc_df = pd.concat([ gfo_bigdf[month] for month in month_list]  )
    gfo_acc_df = gfo_acc_df.reset_index(drop=True)

    

if os.path.exists(gfo_acc_file) :
    print(gfo_acc_file, ' exists' )
    
    
    gfo_acc_df = pd.read_csv(gfo_acc_file, 
                    sep = ',',
                    )

    os.system('bzip2 -v '+ gfo_acc_file)

    
else:
    print(f"---Calculating Grace-FO normalization")
    D500_gfo = normalize_density_msis2( gfo_acc_df , 'GRACE-FO Acc', 500)
    gfo_acc_df['D500_gfo'] = D500_gfo
    #### save to a csv
    gfo_acc_df.to_csv(gfo_acc_file, index=False)  

    

# gfo_df2 =     gfo_df.query("Date >= '2018-10-14' and Date < '2018-12-30'")
# del gfo_df
    

<>:40: SyntaxWarning:

invalid escape sequence '\s'

<>:40: SyntaxWarning:

invalid escape sequence '\s'

/tmp/ipykernel_9984/3599698310.py:40: SyntaxWarning:

invalid escape sequence '\s'

  gfo_acc_june.csv.bz2: 

must bunzip


done


gfo_acc_june.csv  exists


  gfo_acc_june.csv:  3.322:1,  2.408 bits/byte, 69.90% saved, 24257014 in, 7302431 out.


In [5]:
# import sys
# sys.exit(0)

## ICEsat-2 2022

In [6]:
if os.path.exists(ice_pod_file):
    print(ice_pod_file, ' exists' )

    
elif os.path.exists(ice_pod_file+'.bz2') :
    
    print('must bunzip')
    os.system('bunzip2 -v '+ ice_pod_file+'.bz2')
    
else:


    %load_ext autoreload
    %autoreload 2
    from gc import collect as gc_collect
    import pickle 
    from datetime import datetime,timedelta

    scale_cadence = 3

    dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
    run_dict={}
    for imonth,month in enumerate(month_list):
        for i in run_list:
            if i =='msis2':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 5
                run_dict[month+i]['model_path'] = None
            if i =='dtm2020_o':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 3
                run_dict[month+i]['model_path'] = None
            if i =='jb2008':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 1
                run_dict[month+i]['model_path'] = None
            if i =='hasdm_oc':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 2 
                run_dict[month+i]['model_path'] = dir_modeldat+'/hasdm/sethasdm_orbitclouds' #HASDM_OrbitCloud_2018313.01.csv

    print(run_dict)



    dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
                 + 'icesat2/O2R2023_longimeperiod/ScaledDensitiesJune2022/'
    obj = {}
    for imonth,month in enumerate(month_list):
        for i,model in enumerate(run_list):
            pickleName = f'_{month}_DRIA_scale{scale_cadence}.pkl'

            ### Load the data if the pickles exist
            print()
            print()
            gc_collect()

            pickle_file = dir_save+model+pickleName
            
            obj[month+model] = pd.read_pickle(pickle_file)

#             filehandler = open(pickle_file, 'rb') 
#             obj[month+model] = pickle.load(filehandler)
#             filehandler.close()
            print('Loaded data from pickle... ',  month+model)


    ### Save space if doing density retrieval
    for model in run_dict.keys():
        del obj[model]['OrbitResids']
        del obj[model]['Trajectory_orbfil']

    gc_collect()
    
    
    ## MAKE SCALING FACTORS
    
    satid = 1807001
    wgts = {}

    for model in run_dict.keys():
        wgts[model] = {}
        ScalingFactors  = []
        ScalingFactor_times = []

        for ii,arc in enumerate(obj[model]['global_params']['arc_input']):
            epochstart = obj[model]['global_params']['prms']['epoch_start'][ii]
            hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
            frachours =(hrs/24)
            #
            if len(arc) == 9:
                maneuv_indicator = arc[8]
            else:
                maneuv_indicator = ''
            arc_type = obj[model]['global_params']['prms']['arc_type']
            if arc_type == "Nominal30hr_and_AB":
                arc_name =arc[:8]+ maneuv_indicator
            else:
                arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
            ### Collect the weights for the ensemble average
            inv_rms          = 1/obj[model]['Statistics'][arc_name]['T_RMS'].values[0]
            wgts[model][arc_name] = inv_rms#/sum_wgts

            iters = int(obj[model]['run_parameters'+arc_name]['total_iterations']) 
            for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
                if iit == 0 or iit==9:
                    pass
                else:
                    CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
                    APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
                    ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
                    ScalingFactor_times.append(itime)
        run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
        run_dict[model]['ScalingFactors']      = ScalingFactors
        run_dict[model]['Weight'] = wgts[model]



    ###### Scale the densities
    models_dens = {}
    for monthmodel in run_dict.keys():
        print(f"---Making continuous scaled rho for {monthmodel}")
        models_dens[monthmodel] = get_continuous_scaled_densities(obj, run_dict, monthmodel, scale_cadence)

        del obj[monthmodel]
        
    gc_collect()               
    ## Retrieve scaled ensemble weighted average
    ##     'Rho_x' denotes the ensemble weighted avg
    print(f"---Making ensemble avg")
#     for monthmodel in run_dict.keys():
#         month = monthmodel[:3]
    models_dens =  calc_rho_ScaledEnsembleWgtAvg(models_dens, run_dict, month_list, run_list)


    ### Clear up space 
    del obj
    for monthmodel in run_dict.keys():
        print(monthmodel)
        if 'Rho_x' not in monthmodel:
            del models_dens[monthmodel]

    gc_collect()      
    
    
    
if os.path.exists(ice_pod_file) :
    print(ice_pod_file, ' exists' )
    
    ice_pod_df = pd.read_csv(ice_pod_file, 
                    sep = ',',
                    )

#     os.system('bzip2 -v '+ gfo_pod_df)

else:
    
    ice_pod_bigdf = {}
    print(f"---Calculating ICE_pod normalization")
    
    for imonth,month in enumerate(month_list):
        print(month)
        
        D500_ice_pod = normalize_density_msis2( models_dens[month+'Rho_x'], 'ICESat-2', 500)
        models_dens[month+'Rho_x']['D500_ice_pod'] = D500_ice_pod
        
        ice_pod_bigdf[month] = pd.DataFrame.from_dict(models_dens[month+'Rho_x'])
        
        del models_dens[month+'Rho_x']

    print('--concatenating and saving')

    ice_pod_df = pd.concat([ ice_pod_bigdf[month] for month in month_list]  )
    ice_pod_df = ice_pod_df.reset_index(drop=True)
    ice_pod_df.to_csv(ice_pod_file, index=False)  

    ### Need to set this up to only write to file once. and have that write include the full dataset. Maybe concat the dataframes by month first



NameError: name 'run_list' is not defined

In [ ]:
# import pandas as pd
# from datetime import datetime,timedelta
# import numpy as np

# timestart = pd.to_datetime('2022-06-05 00:00:00', format='%Y-%m-%d %H:%M:%S')
# timeend   = pd.to_datetime('2022-06-08 00:00:00', format='%Y-%m-%d %H:%M:%S')

# arcs_yyyyddd         = []
# epochstart = []
# epochstop   = []

# itime = timestart
# while itime < timeend:
               
#     itime_0 = itime
#     itime = itime + pd.to_timedelta(24,'h')
    
#     arcs_yyyyddd.append(itime_0.strftime('%Y.%j'))
#     epochstart.append(itime_0.strftime('%Y-%m-%d %H:%M:%S'))
#     epochstop.append(itime.strftime(  '%Y-%m-%d %H:%M:%S'))

# #     print(' ',itime_0.strftime('%Y.%j'),itime_0, ' to ', itime)        


# ##### update the epoch start to be 3-hours before and epoch end to be 3 hours later
# input_arcs       = []
# input_epochstart = []
# input_epochstop   = []


# for i in epochstart:
#     itime = pd.to_datetime(i) - pd.to_timedelta(3,'h')
#     input_epochstart.append(itime.strftime('%Y-%m-%d %H:%M:%S'))

# for i in epochstop:
#     itime = pd.to_datetime(i) + pd.to_timedelta(3,'h')
#     input_epochstop.append(itime.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from pygeodyn.PYGEODYN import Pygeodyn
import datetime


scaling_cadence = 3
scale_cadence = scaling_cadence
index_buffarc_start = 0
run_list = [
                'jb2008',
                'msis2',
                'dtm2020_o',
#                 'hasdm_oc'
           ]

month_list = ['jun']

dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
run_dict={}
for i in run_list:
    if i =='msis2':
        run_dict[i]={}
        run_dict[i]['num'] = 5
        run_dict[i]['model_path'] = None
    if i =='dtm2020_o':
        run_dict[i]={}
        run_dict[i]['num'] = 3
        run_dict[i]['model_path'] = None
    if i =='jb2008':
        run_dict[i]={}
        run_dict[i]['num'] = 1
        run_dict[i]['model_path'] = None
    if i =='hasdm_oc':
        run_dict[i]={}
        run_dict[i]['num'] = 2 
        run_dict[i]['model_path'] = dir_modeldat+'/hasdm/sethasdm_orbitclouds' #HASDM_OrbitCloud_2018313.01.csv
            
print(run_dict)

In [ ]:
# run_dict['jb2008']

## Spire POD

### Spire 113

In [ ]:
satnum = '113'

if os.path.exists(sp113_pod_file):
    print(sp113_pod_file, ' exists' )

    
elif os.path.exists(sp113_pod_file+'.bz2') :
    
    print('must bunzip')
    os.system('bunzip2 -v '+ sp113_pod_file+'.bz2')
    
else:


    %load_ext autoreload
    %autoreload 2
    from gc import collect as gc_collect
    import pickle 
    from datetime import datetime,timedelta


    dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
    run_dict={}
    for imonth,month in enumerate(month_list):
        for i in run_list:
            if i =='msis2':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 5
                run_dict[month+i]['model_path'] = None
            if i =='dtm2020_o':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 3
                run_dict[month+i]['model_path'] = None
            if i =='jb2008':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 1
                run_dict[month+i]['model_path'] = None
            if i =='hasdm_oc':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 2 
                run_dict[month+i]['model_path'] = dir_modeldat+'/hasdm/sethasdm_orbitclouds' 

    print(run_dict)



    dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
                 + 'spire'+satnum+'/'
    obj = {}
    for imonth,month in enumerate(month_list):
        for i,model in enumerate(run_list):
            pickleName = f'_DRIA_scale{scale_cadence}.pkl'

            ### Load the data if the pickles exist
            print()
            print()
            gc_collect()

            pickle_file = dir_save+model+pickleName
            
            obj[month+model] = pd.read_pickle(pickle_file)

#             filehandler = open(pickle_file, 'rb') 
#             obj[month+model] = pickle.load(filehandler)
#             filehandler.close()
            print('Loaded data from pickle... ',  month+model)


    ### Save space if doing density retrieval
    for model in run_dict.keys():
        del obj[model]['OrbitResids']
        del obj[model]['Trajectory_orbfil']

    gc_collect()
    
    
    ## MAKE SCALING FACTORS
    
    if int(satnum) == 106:
        satid=1903826
    elif int(satnum) == 104:
        satid = 1903812
    elif int(satnum) == 113:
        satid = 1903824

    else:
        print('Add satellite Numbers')

        import sys
        sys.exit(0)

    
    wgts = {}

    for model in run_dict.keys():
        wgts[model] = {}
        ScalingFactors  = []
        ScalingFactor_times = []

        for ii,arc in enumerate(obj[model]['global_params']['arc_input']):
            epochstart = obj[model]['global_params']['prms']['epoch_start'][ii]
            hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
            frachours =(hrs/24)
            #
            if len(arc) == 9:
                maneuv_indicator = arc[8]
            else:
                maneuv_indicator = ''
            arc_type = obj[model]['global_params']['prms']['arc_type']
            if arc_type == "Nominal30hr_and_AB":
                arc_name =arc[:8]+ maneuv_indicator
            else:
                arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
            ### Collect the weights for the ensemble average
            inv_rms          = 1/obj[model]['Statistics'][arc_name]['T_RMS'].values[0]
            wgts[model][arc_name] = inv_rms#/sum_wgts

            iters = int(obj[model]['run_parameters'+arc_name]['total_iterations']) 
            for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
                if iit == 0 or iit==9:
#                     print(itime)
                    pass
                else:
                    CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
                    APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
                    ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
                    ScalingFactor_times.append(itime)
        if int(satnum) == 113:
            data = np.sort(ScalingFactors)

            ### Calculate the first quartile (Q1) and third quartile (Q3)
            Q1 = np.percentile(data, 25)
            Q3 = np.percentile(data, 75)
            median = np.percentile(data, 50)
            ### Calculate the IQR
            IQR = Q3 - Q1

            ### Define a threshold for identifying outliers
            threshold = 1                                ## You can adjust this threshold as needed

            ### Calculate the lower and upper bounds for outliers
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR

            df = pd.DataFrame(data={'x':ScalingFactor_times,
                                    'y':ScalingFactors})
            filtered_df = df[(df['y'] >= lower_bound) & (df['y'] <= upper_bound)]

            ScalingFactor_times = pd.to_datetime(filtered_df['x'].values)
            ScalingFactors      = filtered_df['y'].values

        run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
        run_dict[model]['ScalingFactors']      = ScalingFactors
        run_dict[model]['Weight'] = wgts[model]
        

    ###### Scale the densities
    models_dens = {}
    for monthmodel in run_dict.keys():
        print(f"---Making continuous scaled rho for {monthmodel}")
        models_dens[monthmodel] = get_continuous_scaled_densities(obj, run_dict, monthmodel, scale_cadence)

#         del obj[monthmodel]
        
    gc_collect()               
    ## Retrieve scaled ensemble weighted average
    ##     'Rho_x' denotes the ensemble weighted avg
    print(f"---Making ensemble avg")
#     for monthmodel in run_dict.keys():
#         month = monthmodel[:3]
    models_dens =  calc_rho_ScaledEnsembleWgtAvg(models_dens, run_dict, month_list, run_list)


    ### Clear up space 
#     del obj
    for monthmodel in run_dict.keys():
        print(monthmodel)
        if 'Rho_x' not in monthmodel:
            del models_dens[monthmodel]

    gc_collect()      
    
    
    
if os.path.exists(sp113_pod_file) :
    print(sp113_pod_file, ' exists' )
    
    sp113_pod_df = pd.read_csv(sp113_pod_file, 
                    sep = ',',
                    )

#     os.system('bzip2 -v '+ gfo_pod_df)

else:
    
    sp113_pod_bigdf = {}
    print(f"---Calculating Spire {satnum} pod normalization")
    
    for imonth,month in enumerate(month_list):
        print(month)
        
        D500_sp113_pod = normalize_density_msis2( models_dens[month+'Rho_x'], 'Spire '+satnum, 500)
        models_dens[month+'Rho_x']['D500_sp113_pod'] = D500_sp113_pod
        
        sp113_pod_bigdf[month] = pd.DataFrame.from_dict(models_dens[month+'Rho_x'])
        
        del models_dens[month+'Rho_x']

    print('--concatenating and saving')

    sp113_pod_df = pd.concat([ sp113_pod_bigdf[month] for month in month_list]  )
    sp113_pod_df = sp113_pod_df.reset_index(drop=True)
    sp113_pod_df.to_csv(sp113_pod_file, index=False)  

    ### Need to set this up to only write to file once. and have that write include the full dataset. Maybe concat the dataframes by month first



### Spire 106

In [ ]:
satnum = '106'

if os.path.exists(sp106_pod_file):
    print(sp106_pod_file, ' exists' )

    
elif os.path.exists(sp106_pod_file+'.bz2') :
    
    print('must bunzip')
    os.system('bunzip2 -v '+ sp106_pod_file+'.bz2')
    
else:


    %load_ext autoreload
    %autoreload 2
    from gc import collect as gc_collect
    import pickle 
    from datetime import datetime,timedelta


    dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
    run_dict={}
    for imonth,month in enumerate(month_list):
        for i in run_list:
            if i =='msis2':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 5
                run_dict[month+i]['model_path'] = None
            if i =='dtm2020_o':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 3
                run_dict[month+i]['model_path'] = None
            if i =='jb2008':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 1
                run_dict[month+i]['model_path'] = None
            if i =='hasdm_oc':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 2 
                run_dict[month+i]['model_path'] = dir_modeldat+'/hasdm/sethasdm_orbitclouds' 

    print(run_dict)



    dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
                 + 'spire'+satnum+'/'
    obj = {}
    for imonth,month in enumerate(month_list):
        for i,model in enumerate(run_list):
            pickleName = f'_DRIA_scale{scale_cadence}.pkl'

            ### Load the data if the pickles exist
            print()
            print()
            gc_collect()

            pickle_file = dir_save+model+pickleName
            
            obj[month+model] = pd.read_pickle(pickle_file)

#             filehandler = open(pickle_file, 'rb') 
#             obj[month+model] = pickle.load(filehandler)
#             filehandler.close()
            print('Loaded data from pickle... ',  month+model)


    ### Save space if doing density retrieval
    for model in run_dict.keys():
        del obj[model]['OrbitResids']
        del obj[model]['Trajectory_orbfil']

    gc_collect()
    
    
    ## MAKE SCALING FACTORS
    
    if int(satnum) == 106:
        satid=1903826
    elif int(satnum) == 104:
        satid = 1903812
    elif int(satnum) == 113:
        satid = 1903824

    else:
        print('Add satellite Numbers')

        import sys
        sys.exit(0)

    
    wgts = {}

    for model in run_dict.keys():
        wgts[model] = {}
        ScalingFactors  = []
        ScalingFactor_times = []

        for ii,arc in enumerate(obj[model]['global_params']['arc_input']):
            epochstart = obj[model]['global_params']['prms']['epoch_start'][ii]
            hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
            frachours =(hrs/24)
            #
            if len(arc) == 9:
                maneuv_indicator = arc[8]
            else:
                maneuv_indicator = ''
            arc_type = obj[model]['global_params']['prms']['arc_type']
            if arc_type == "Nominal30hr_and_AB":
                arc_name =arc[:8]+ maneuv_indicator
            else:
                arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
            ### Collect the weights for the ensemble average
            inv_rms          = 1/obj[model]['Statistics'][arc_name]['T_RMS'].values[0]
            wgts[model][arc_name] = inv_rms#/sum_wgts

            iters = int(obj[model]['run_parameters'+arc_name]['total_iterations']) 
            for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
                if iit == 0 or iit==9:
#                     print(itime)
                    pass
                else:
                    CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
                    APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
                    ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
                    ScalingFactor_times.append(itime)
        if int(satnum) == 106:
            data = np.sort(ScalingFactors)

            ### Calculate the first quartile (Q1) and third quartile (Q3)
            Q1 = np.percentile(data, 25)
            Q3 = np.percentile(data, 75)
            median = np.percentile(data, 50)
            ### Calculate the IQR
            IQR = Q3 - Q1

            ### Define a threshold for identifying outliers
            threshold = 1                                ## You can adjust this threshold as needed

            ### Calculate the lower and upper bounds for outliers
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR

            df = pd.DataFrame(data={'x':ScalingFactor_times,
                                    'y':ScalingFactors})
            filtered_df = df[(df['y'] >= lower_bound) & (df['y'] <= upper_bound)]

            ScalingFactor_times = pd.to_datetime(filtered_df['x'].values)
            ScalingFactors      = filtered_df['y'].values

        run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
        run_dict[model]['ScalingFactors']      = ScalingFactors
        run_dict[model]['Weight'] = wgts[model]
        

    ###### Scale the densities
    models_dens = {}
    for monthmodel in run_dict.keys():
        print(f"---Making continuous scaled rho for {monthmodel}")
        models_dens[monthmodel] = get_continuous_scaled_densities(obj, run_dict, monthmodel, scale_cadence)

#         del obj[monthmodel]
        
    gc_collect()               
    ## Retrieve scaled ensemble weighted average
    ##     'Rho_x' denotes the ensemble weighted avg
    print(f"---Making ensemble avg")
#     for monthmodel in run_dict.keys():
#         month = monthmodel[:3]
    models_dens =  calc_rho_ScaledEnsembleWgtAvg(models_dens, run_dict, month_list, run_list)


    ### Clear up space 
#     del obj
    for monthmodel in run_dict.keys():
        print(monthmodel)
        if 'Rho_x' not in monthmodel:
            del models_dens[monthmodel]

    gc_collect()      
    
    
    
if os.path.exists(sp106_pod_file) :
    print(sp106_pod_file, ' exists' )
    
    sp106_pod_df = pd.read_csv(sp106_pod_file, 
                    sep = ',',
                    )

#     os.system('bzip2 -v '+ gfo_pod_df)

else:
    
    sp106_pod_bigdf = {}
    print(f"---Calculating Spire {satnum} pod normalization")
    
    for imonth,month in enumerate(month_list):
        print(month)
        
        D500_sp106_pod = normalize_density_msis2( models_dens[month+'Rho_x'], 'Spire '+satnum, 500)
        models_dens[month+'Rho_x']['D500_sp106_pod'] = D500_sp106_pod
        
        sp106_pod_bigdf[month] = pd.DataFrame.from_dict(models_dens[month+'Rho_x'])
        
        del models_dens[month+'Rho_x']

    print('--concatenating and saving')

    sp106_pod_df = pd.concat([ sp106_pod_bigdf[month] for month in month_list]  )
    sp106_pod_df = sp106_pod_df.reset_index(drop=True)
    sp106_pod_df.to_csv(sp106_pod_file, index=False)  

    ### Need to set this up to only write to file once. and have that write include the full dataset. Maybe concat the dataframes by month first



### Spire 104

In [ ]:
satnum = '104'
if os.path.exists(sp104_pod_file):
    print(sp104_pod_file, ' exists' )

    
elif os.path.exists(sp104_pod_file+'.bz2') :
    
    print('must bunzip')
    os.system('bunzip2 -v '+ sp104_pod_file+'.bz2')
    
else:


    %load_ext autoreload
    %autoreload 2
    from gc import collect as gc_collect
    import pickle 
    from datetime import datetime,timedelta


    dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
    run_dict={}
    for imonth,month in enumerate(month_list):
        for i in run_list:
            if i =='msis2':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 5
                run_dict[month+i]['model_path'] = None
            if i =='dtm2020_o':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 3
                run_dict[month+i]['model_path'] = None
            if i =='jb2008':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 1
                run_dict[month+i]['model_path'] = None
            if i =='hasdm_oc':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 2 
                run_dict[month+i]['model_path'] = dir_modeldat+'/hasdm/sethasdm_orbitclouds' #HASDM_OrbitCloud_2018313.01.csv

    print(run_dict)



    dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
                 + 'spire104/'
    obj = {}
    for imonth,month in enumerate(month_list):
        for i,model in enumerate(run_list):
            pickleName = f'_DRIA_scale{scale_cadence}.pkl'

            ### Load the data if the pickles exist
            print()
            print()
            gc_collect()

            pickle_file = dir_save+model+pickleName
            
            obj[month+model] = pd.read_pickle(pickle_file)

#             filehandler = open(pickle_file, 'rb') 
#             obj[month+model] = pickle.load(filehandler)
#             filehandler.close()
            print('Loaded data from pickle... ',  month+model)


    ### Save space if doing density retrieval
    for model in run_dict.keys():
        del obj[model]['OrbitResids']
        del obj[model]['Trajectory_orbfil']

    gc_collect()
    
    
    ## MAKE SCALING FACTORS
    
    satid = 1903812
    wgts = {}

    for model in run_dict.keys():
        wgts[model] = {}
        ScalingFactors  = []
        ScalingFactor_times = []

        for ii,arc in enumerate(obj[model]['global_params']['arc_input']):
            epochstart = obj[model]['global_params']['prms']['epoch_start'][ii]
            hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
            frachours =(hrs/24)
            #
            if len(arc) == 9:
                maneuv_indicator = arc[8]
            else:
                maneuv_indicator = ''
            arc_type = obj[model]['global_params']['prms']['arc_type']
            if arc_type == "Nominal30hr_and_AB":
                arc_name =arc[:8]+ maneuv_indicator
            else:
                arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
            ### Collect the weights for the ensemble average
            inv_rms          = 1/obj[model]['Statistics'][arc_name]['T_RMS'].values[0]
            wgts[model][arc_name] = inv_rms#/sum_wgts

            iters = int(obj[model]['run_parameters'+arc_name]['total_iterations']) 
            for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
                if iit == 0 or iit==9:
                    pass
                else:
                    CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
                    APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
                    ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
                    ScalingFactor_times.append(itime)
        if int(satnum) == 104:
            data = np.sort(ScalingFactors)

            ### Calculate the first quartile (Q1) and third quartile (Q3)
            Q1 = np.percentile(data, 25)
            Q3 = np.percentile(data, 75)
            median = np.percentile(data, 50)
            ### Calculate the IQR
            IQR = Q3 - Q1

            ### Define a threshold for identifying outliers
            threshold = 1                                ## You can adjust this threshold as needed

            ### Calculate the lower and upper bounds for outliers
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR

            df = pd.DataFrame(data={'x':ScalingFactor_times,
                                    'y':ScalingFactors})
            filtered_df = df[(df['y'] >= lower_bound) & (df['y'] <= upper_bound)]

            ScalingFactor_times = pd.to_datetime(filtered_df['x'].values)
            ScalingFactors      = filtered_df['y'].values

        run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
        run_dict[model]['ScalingFactors']      = ScalingFactors
        run_dict[model]['Weight'] = wgts[model]

        run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
        run_dict[model]['ScalingFactors']      = ScalingFactors
        run_dict[model]['Weight'] = wgts[model]



    ###### Scale the densities
    models_dens = {}
    for monthmodel in run_dict.keys():
        print(f"---Making continuous scaled rho for {monthmodel}")
        models_dens[monthmodel] = get_continuous_scaled_densities(obj, run_dict, monthmodel, scale_cadence)

        del obj[monthmodel]
        
    gc_collect()               
    ## Retrieve scaled ensemble weighted average
    ##     'Rho_x' denotes the ensemble weighted avg
    print(f"---Making ensemble avg")
#     for monthmodel in run_dict.keys():
#         month = monthmodel[:3]
    models_dens =  calc_rho_ScaledEnsembleWgtAvg(models_dens, run_dict, month_list, run_list)


    ### Clear up space 
    del obj
    for monthmodel in run_dict.keys():
        print(monthmodel)
        if 'Rho_x' not in monthmodel:
            del models_dens[monthmodel]

    gc_collect()      
    
    
    
if os.path.exists(sp104_pod_file) :
    print(sp104_pod_file, ' exists' )
    
    sp104_pod_df = pd.read_csv(sp104_pod_file, 
                    sep = ',',
                    )

#     os.system('bzip2 -v '+ gfo_pod_df)

else:
    
    sp104_pod_bigdf = {}
    print(f"---Calculating Spire 104 pod normalization")
    
    for imonth,month in enumerate(month_list):
        print(month)
        
        D500_sp104_pod = normalize_density_msis2( models_dens[month+'Rho_x'], 'Spire 104', 500)
        models_dens[month+'Rho_x']['D500_sp104_pod'] = D500_sp104_pod
        
        sp104_pod_bigdf[month] = pd.DataFrame.from_dict(models_dens[month+'Rho_x'])
        
        del models_dens[month+'Rho_x']

    print('--concatenating and saving')

    sp104_pod_df = pd.concat([ sp104_pod_bigdf[month] for month in month_list]  )
    sp104_pod_df = sp104_pod_df.reset_index(drop=True)
    sp104_pod_df.to_csv(sp104_pod_file, index=False)  

    ### Need to set this up to only write to file once. and have that write include the full dataset. Maybe concat the dataframes by month first



## Filter for common times

In [ ]:
gfo_acc_df = gfo_acc_df.query("Date >= '2022-06-01' and Date < '2022-06-30'")
sp104_pod_df = sp104_pod_df.query("date >= '2022-06-03' and date < '2022-06-25'")
sp113_pod_df = sp113_pod_df.query("date >= '2022-06-05' and date < '2022-06-28'")
sp106_pod_df = sp106_pod_df.query("date >= '2022-06-04' and date < '2022-06-12'")

(timeavg_gfo_acc,  denavg_gfo_acc) = orbit_avg_generic(gfo_acc_df['Date'], gfo_acc_df['D500_gfo'], gfo_acc_df['lat'])    
(timeavg_sp104_pod,  denavg_sp104_pod) = orbit_avg_generic(sp104_pod_df['date'], sp104_pod_df['D500_sp104_pod'], sp104_pod_df['lat'])    
(timeavg_sp113_pod,  denavg_sp113_pod) = orbit_avg_generic(sp113_pod_df['date'], sp113_pod_df['D500_sp113_pod'], sp113_pod_df['lat'])    
(timeavg_sp106_pod,  denavg_sp106_pod) = orbit_avg_generic(sp106_pod_df['date'], sp106_pod_df['D500_sp106_pod'], sp106_pod_df['lat'])    


In [ ]:

def calc_percent_diff_logspace(model, obs):
    from numpy import exp as np_exp
    from numpy import log as np_log # this is the natural log
    
#     return( ( np_exp(np_log(model/obs))-1 )*100)
    return(  np_exp(np_log(model/obs)))


In [ ]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))


perc_stat = {}
perc_stat['time_midpoint_GFO'] = []
perc_stat['time_midpoint_sp104'] = []
perc_stat['gfo_acc_denorbavg'] = []
perc_stat['sp104_pod_denorbavg'] = []
perc_stat['gfo_acc_time']      = []
perc_stat['sp104_pod_time']      = []
# 
perc_stat['percdiff_GFO_log']    = []
perc_stat['percdiff_sp104_log']    = []
# 
perc_stat['ratio_GFO']    = []
perc_stat['ratio_sp104']    = []


from numpy import exp as np_exp
from numpy import log as np_log # this is the natural log


for i,val in enumerate( timeavg_gfo_acc):
    ###-----------------------------------
    date_near = nearest(timeavg_sp104_pod, val)
    indx = np.where(pd.to_datetime(timeavg_sp104_pod) == pd.to_datetime(date_near))[0][0]
    
    if np.abs((val - timeavg_sp104_pod[indx]).total_seconds()) > 3000:  # only compute % change if values within 1 orbit (95mins)
        pass
    
    else:
        if np.abs((timeavg_sp104_pod[indx] - timeavg_sp104_pod[indx-1]).total_seconds()) > 40000:
            print('skipping ', indx, timeavg_sp104_pod[indx])
        else:
            perc_stat['gfo_acc_time'].append( val)
            perc_stat['gfo_acc_denorbavg'].append( denavg_gfo_acc[i])

            perc_stat['time_midpoint_sp104'].append( pd.Timestamp(val) + (pd.Timestamp(timeavg_sp104_pod[indx]) - pd.Timestamp(val)) / 2)
            perc_stat['sp104_pod_denorbavg'].append( denavg_sp104_pod[indx])
            perc_stat['sp104_pod_time'].append( timeavg_sp104_pod[indx])
            perc_stat['percdiff_sp104_log'].append(calc_percent_diff_logspace(denavg_sp104_pod[indx], denavg_gfo_acc[i]) )
            perc_stat['ratio_sp104'].append(denavg_sp104_pod[indx]/denavg_gfo_acc[i])


In [ ]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

# perc_stat = {}
perc_stat['time_midpoint_GFO'] = []
perc_stat['time_midpoint_sp113'] = []
perc_stat['gfo_acc_denorbavg'] = []
perc_stat['sp113_pod_denorbavg'] = []
perc_stat['gfo_acc_time']      = []
perc_stat['sp113_pod_time']      = []
# 
perc_stat['percdiff_GFO_log']    = []
perc_stat['percdiff_sp113_log']    = []
# 
perc_stat['ratio_GFO']    = []
perc_stat['ratio_sp113']    = []


from numpy import exp as np_exp
from numpy import log as np_log # this is the natural log


for i,val in enumerate( timeavg_gfo_acc):
    ###-----------------------------------
    date_near = nearest(timeavg_sp113_pod, val)
    indx = np.where(pd.to_datetime(timeavg_sp113_pod) == pd.to_datetime(date_near))[0][0]
    
    if np.abs((val - timeavg_sp113_pod[indx]).total_seconds()) > 3000:  # only compute % change if values within 1 orbit (95mins)
        pass
    
    else:
        if np.abs((timeavg_sp113_pod[indx] - timeavg_sp113_pod[indx-1]).total_seconds()) > 40000:
            print('skipping ', indx, timeavg_sp113_pod[indx])
        else:
            perc_stat['gfo_acc_time'].append( val)
            perc_stat['gfo_acc_denorbavg'].append( denavg_gfo_acc[i])

            perc_stat['time_midpoint_sp113'].append( pd.Timestamp(val) + (pd.Timestamp(timeavg_sp113_pod[indx]) - pd.Timestamp(val)) / 2)
            perc_stat['sp113_pod_denorbavg'].append( denavg_sp113_pod[indx])
            perc_stat['sp113_pod_time'].append( timeavg_sp113_pod[indx])
            perc_stat['percdiff_sp113_log'].append(calc_percent_diff_logspace(denavg_sp113_pod[indx], denavg_gfo_acc[i]) )
            perc_stat['ratio_sp113'].append(denavg_sp113_pod[indx]/denavg_gfo_acc[i])


In [ ]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

# perc_stat = {}
perc_stat['time_midpoint_GFO'] = []
perc_stat['time_midpoint_sp106'] = []
perc_stat['gfo_acc_denorbavg'] = []
perc_stat['sp106_pod_denorbavg'] = []
perc_stat['gfo_acc_time']      = []
perc_stat['sp106_pod_time']      = []
# 
perc_stat['percdiff_GFO_log']    = []
perc_stat['percdiff_sp106_log']    = []
# 
perc_stat['ratio_GFO']    = []
perc_stat['ratio_sp106']    = []


from numpy import exp as np_exp
from numpy import log as np_log # this is the natural log


for i,val in enumerate( timeavg_gfo_acc):
    ###-----------------------------------
    date_near = nearest(timeavg_sp106_pod, val)
    indx = np.where(pd.to_datetime(timeavg_sp106_pod) == pd.to_datetime(date_near))[0][0]
    
    if np.abs((val - timeavg_sp106_pod[indx]).total_seconds()) > 3000:  # only compute % change if values within 1 orbit (95mins)
        pass
    
    else:
        if np.abs((timeavg_sp106_pod[indx] - timeavg_sp106_pod[indx-1]).total_seconds()) > 40000:
            print('skipping ', indx, timeavg_sp106_pod[indx])
        else:
            perc_stat['gfo_acc_time'].append( val)
            perc_stat['gfo_acc_denorbavg'].append( denavg_gfo_acc[i])

            perc_stat['time_midpoint_sp106'].append( pd.Timestamp(val) + (pd.Timestamp(timeavg_sp106_pod[indx]) - pd.Timestamp(val)) / 2)
            perc_stat['sp106_pod_denorbavg'].append( denavg_sp106_pod[indx])
            perc_stat['sp106_pod_time'].append( timeavg_sp106_pod[indx])
            perc_stat['percdiff_sp106_log'].append(calc_percent_diff_logspace(denavg_sp106_pod[indx], denavg_gfo_acc[i]) )
            perc_stat['ratio_sp106'].append(denavg_sp106_pod[indx]/denavg_gfo_acc[i])


## Load Kp and Flux

In [ ]:
import linecache
import pandas as pd
import numpy as np

path_to_flux = "/data/SatDragModelValidation/src/geodyn_code/support/geodyn_support/make_tables/"


dateS   = []
dateAP  = []
dateKP8 = []

fluxS   = []
fluxAP  = []
fluxKP8 = []

switch = 0
with open(path_to_flux+"new_master", 'r') as f:
    for line_no, string in enumerate(f):
#         string = linecache.getline("new_master",line+1) #

        if 'MASTER' in string:        
            switch = 'skip'
            continue
            
        ### Skip the title lines and any dates before Y2K
        if 'FLUXS' in string:        
            switch = 'readfluxS'
            print(switch)
            continue

        if 'FLUXAP' in string:        
            switch = 'readfluxAP'
            print(switch)
            continue
        if 'FLUXKP8' in string:        
            switch = 'readfluxKP8'
            print(switch)
            continue

        if switch =='skip':
            continue
            
        if int(string[:2]) >= 58:
            continue
        else:
            
            if int(string[:2]) < 22 or int(string[:2]) > 22:
                continue
                
            if switch == 'readfluxS':
                ### Load the Solar Flux values after 2000. The date leading each line represents
                ### the first flux value in that line.  The following n values are that day + n days.
                ###  All Solar Flux values are mulitplied by 10 for some reason...

                for n in range(0,12):
                    dateS.append(pd.to_datetime(string[:6],format='%y%m%d') + n*pd.to_timedelta(1,'d') )
                fluxS.append(int(string[11:15]) / 10)
                fluxS.append(int(string[16:20]) / 10)
                fluxS.append(int(string[21:25]) / 10)
                fluxS.append(int(string[26:30]) / 10)
                fluxS.append(int(string[31:35]) / 10)
                fluxS.append(int(string[36:40]) / 10)
                fluxS.append(int(string[41:45]) / 10)
                fluxS.append(int(string[46:50]) / 10)
                fluxS.append(int(string[51:55]) / 10)
                fluxS.append(int(string[56:60]) / 10)
                fluxS.append(int(string[61:65]) / 10)
                fluxS.append(int(string[66:70]) / 10)

            if switch == 'readfluxAP':
                ### Load the AP values after 2000. These are formatted the same as the FLUXS section
                for n in range(0,12):
                    dateAP.append(pd.to_datetime(string[:6],format='%y%m%d') + n*pd.to_timedelta(1,'d') )
                fluxAP.append(int(string[11:15]) / 10)
                fluxAP.append(int(string[16:20]) / 10)
                fluxAP.append(int(string[21:25]) / 10)
                fluxAP.append(int(string[26:30]) / 10)
                fluxAP.append(int(string[31:35]) / 10)
                fluxAP.append(int(string[36:40]) / 10)
                fluxAP.append(int(string[41:45]) / 10)
                fluxAP.append(int(string[46:50]) / 10)
                fluxAP.append(int(string[51:55]) / 10)
                fluxAP.append(int(string[56:60]) / 10)
                fluxAP.append(int(string[61:65]) / 10)
                fluxAP.append(int(string[66:70]) / 10)

            if switch == 'readfluxKP8':
                ### Load the KP values after 2000. These are formatted such that each line is a day
                ### and each nth value in the line is n*3hrs for the day
                for n in range(0,8):
                    dateKP8.append(pd.to_datetime(string[:6],format='%y%m%d') + n*pd.to_timedelta(3,'h') )
                fluxKP8.append(int(string[7:11])  / 100)
                fluxKP8.append(int(string[12:16]) / 100)
                fluxKP8.append(int(string[17:21]) / 100)
                fluxKP8.append(int(string[22:26]) / 100)
                fluxKP8.append(int(string[27:31]) / 100)
                fluxKP8.append(int(string[32:36]) / 100)
                fluxKP8.append(int(string[37:41]) / 100)
                fluxKP8.append(int(string[42:46]) / 100)

fluxS = np.array(fluxS)
fluxAP = np.array(fluxAP)
fluxKP8 = np.array(fluxKP8)
                
fluxS   = np.where(fluxS  == 0., np.nan, fluxS  )
fluxAP  = np.where(fluxAP ==0., np.nan, fluxAP )
fluxKP8 = np.where(fluxKP8==0., np.nan, fluxKP8)



In [ ]:
%load_ext autoreload
%autoreload 2


fig = make_subplots(rows=2, cols=1, row_heights=[0.2, 0.8],#],
                    specs=[[{"secondary_y": True}],
#                            [{"secondary_y": False}],
#                            [{"secondary_y": False}],
#                            [{"secondary_y": False}],
                           [{"secondary_y": False}]],
                           shared_xaxes=True,
                           vertical_spacing=0.02)
    
SHOW_alldata = True

#     print('model_m1', model_m1)


fig.add_trace(go.Scattergl(x=dateS,
                           y=fluxS,
                           name= 'F107d_1AU',
                           mode='lines',
                           opacity=1,
                           line = dict(shape = 'hvh',dash='dash', color = 'blue', width=2),
                           showlegend=False),
                           secondary_y=True,row=1, col=1)

fig.add_trace(go.Scattergl(x=dateKP8,
                           y=fluxKP8,
                           name= 'Kp',
                           mode='lines',
                           opacity=1,
                           line = dict(shape = 'hvh', color = 'black', width=2),
                           showlegend=False),
                           secondary_y=False,row=1, col=1) 



fig.add_trace(go.Scattergl(x=timeavg_gfo_acc,
                           y=denavg_gfo_acc,
                           ### name= model_m1,
                           mode='markers+lines',
                           opacity=1,
                               marker=dict(color='black',size=2),
                               line = dict(dash ='solid', color = 'black', width=2),
                           showlegend=False), row=2, col=1)


fig.add_trace(go.Scattergl(x=timeavg_sp104_pod,
                           y=denavg_sp104_pod,
                           ### name= model_m1,
                           mode='markers+lines',
                           opacity=1,
                               marker=dict(size=2),
                               line = dict(dash ='solid',  width=2),
                           showlegend=False), row=2, col=1)

fig.add_trace(go.Scattergl(x=timeavg_sp113_pod,
                           y=denavg_sp113_pod,
                           ### name= model_m1,
                           mode='markers+lines',
                           opacity=1,
                               marker=dict(size=2),
                               line = dict(dash ='solid', width=2),
                           showlegend=False), row=2, col=1)

fig.add_trace(go.Scattergl(x=timeavg_sp106_pod,
                           y=denavg_sp106_pod,
                           ### name= model_m1,
                           mode='markers+lines',
                           opacity=1,
                               marker=dict(size=2),
                               line = dict(dash ='solid', width=2),
                           showlegend=False), row=2, col=1)

    
### UPDATE AXES 
fig.update_yaxes(title_text="Kp", 
                 exponentformat= 'power',
#                  range=[0,7],
                 secondary_y=False,
                 row=1, col=1)
fig.update_yaxes(title_text="F10.7", 
                 exponentformat= 'power',
                 range=[35,170],
                 secondary_y=True,
                 tickfont=dict(color="blue"),
                 titlefont=dict(color="blue"),
                 row=1, col=1)
fig.update_xaxes(range=[pd.to_datetime('2022-06-01 02:00:00'), pd.to_datetime('2022-06-30 00:00:00')])




################################################################################
    
#######################################################
font_dict=dict(family='Arial',size=11,color='black')


fig = plot_cleanformat_axes(fig, font_dict)

fig.update_yaxes(title_text="Density", 
                 type="log", 
                 exponentformat= 'power',row=2, col=1)

    
# a='input'
# s=settings_spire
fig.update_layout(title=f"Density Validation- Spire and GFO", 
#                   title=f"{s['cd_model'][a]}, {s['hours_between_cd_adj'][a]}-hr Scale from CD={s['cd_value'][a]}", 
                  autosize=False,    width=1000,    height=800,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.update_annotations(font_size=16)  # Increase size of subplot title
# fig.show(config=config)
fig.show(config= dict({
                'displayModeBar': False,
                'responsive': True,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                }),
#          renderer='jpg',
)




In [ ]:

gfopodcolor = "#d62728"
sp104podcolor = "#AB63FA"
sp106podcolor = "cyan"
gfoacccolor = 'black'
sp113podcolor = '#FF6692'

# fig = make_subplots(rows=1, cols=1)
fig = make_subplots(rows=3, cols=1, row_heights=[0.2, 0.5, 0.3 ],#],
                    specs=[[{"secondary_y": True}],
                           [{"secondary_y": False}],
#                            [{"secondary_y": False}],
#                            [{"secondary_y": False}],
                           [{"secondary_y": False}]],
                           shared_xaxes=True,
                           vertical_spacing=0.02)

fig.add_trace(go.Scatter(x=dateS,
                           y=fluxS,
                           name= 'F107d_1AU',
                           mode='lines',
                           opacity=1,
                           line = dict(shape = 'hvh',dash='dash', color = 'blue', width=2),
                           showlegend=False),
                           secondary_y=True,row=1, col=1)

fig.add_trace(go.Scatter(x=dateKP8,
                           y=fluxKP8,
                           name= 'Kp',
                           mode='lines',
                           opacity=1,
                           line = dict(shape = 'hvh', color = 'grey', width=2),
                           showlegend=False),
                           secondary_y=False,row=1, col=1) 



# fig.add_trace(go.Scattergl(x= perc_stat['gfo_acc_time'],
#                             y=perc_stat['gfo_acc_denorbavg'],
#                             name=f'GFO_acc_500km',
#                             mode='markers',
#                             opacity=1,
#                             marker=dict( size=4, color=gfoacccolor),
#                             showlegend=False),
#                             row=2, col=1)
fig.add_trace(go.Scattergl(x= timeavg_gfo_acc,
                            y=denavg_gfo_acc,
                            name=f'GFO_acc_500km',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=4, color=gfoacccolor),
                            showlegend=False),
                            row=2, col=1)



fig.add_trace(go.Scattergl(x=perc_stat['sp104_pod_time'],
                            y=perc_stat['sp104_pod_denorbavg'],
                            name=f'D500_sp104_pod',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=4, color=sp104podcolor),
                            showlegend=False),
                            row=2, col=1)
fig.add_trace(go.Scattergl(x=perc_stat['time_midpoint_sp104'],
                            y=perc_stat['percdiff_sp104_log'],
#                             name=f'D500_icesat2',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=3, color=sp104podcolor),
                            showlegend=False),
                            row=3, col=1)


fig.add_trace(go.Scattergl(x=perc_stat['sp113_pod_time'],
                            y=perc_stat['sp113_pod_denorbavg'],
                            name=f'D500_sp113_pod',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=4, color=sp113podcolor),
                            showlegend=False),
                            row=2, col=1)

fig.add_trace(go.Scattergl(x=perc_stat['time_midpoint_sp113'],
                            y=perc_stat['percdiff_sp113_log'],
#                             name=f'D500_icesat2',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=3, color=sp113podcolor),
                            showlegend=False),
                            row=3, col=1)


fig.add_trace(go.Scattergl(x=perc_stat['sp106_pod_time'],
                            y=perc_stat['sp106_pod_denorbavg'],
                            name=f'D500_sp106_pod',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=4, color=sp106podcolor),
                            showlegend=False),
                            row=2, col=1)
fig.add_trace(go.Scattergl(x=perc_stat['time_midpoint_sp106'],
                            y=perc_stat['percdiff_sp106_log'],
#                             name=f'D500_icesat2',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=3, color=sp106podcolor),
                            showlegend=False),
                            row=3, col=1)


fig.add_hline(y=0, line = dict(dash='solid', color = 'grey', width=1), row=3, col=1)

df = pd.DataFrame(data={'time':perc_stat['time_midpoint_sp104'],'percdiff_sp104_log': perc_stat['percdiff_sp104_log'] })
fig.add_trace(go.Scattergl(x=perc_stat['time_midpoint_sp104'],
                            y=df.rolling(window=16, center=True)['percdiff_sp104_log'].mean(),
                            opacity=1,
#                             marker=dict( size=2, color='#1f77b4'),
                           line = dict( color =sp104podcolor, width=4),
                            showlegend=False),
                            row=3, col=1)

df = pd.DataFrame(data={'time':perc_stat['time_midpoint_sp113'],'percdiff_sp113_log': perc_stat['percdiff_sp113_log'] })
fig.add_trace(go.Scattergl(x=perc_stat['time_midpoint_sp113'],
                            y=df.rolling(window=16, center=True)['percdiff_sp113_log'].mean(),
                            opacity=1,
#                             marker=dict( size=2, color='#1f77b4'),
                           line = dict( color =sp113podcolor, width=4),
                            showlegend=False),
                            row=3, col=1)

df = pd.DataFrame(data={'time':perc_stat['time_midpoint_sp106'],'percdiff_sp106_log': perc_stat['percdiff_sp106_log'] })
fig.add_trace(go.Scattergl(x=perc_stat['time_midpoint_sp106'],
                            y=df.rolling(window=16, center=True)['percdiff_sp106_log'].mean(),
                            opacity=1,
#                             marker=dict( size=2, color='#1f77b4'),
                           line = dict( color =sp106podcolor, width=4),
                            showlegend=False),
                            row=3, col=1)

#### FANCY LEGEND ################################################################
modelnames=[]
modelcolors = []
modelnames.append("GraceFO Acc")
modelcolors.append("black")
# modelnames.append("GraceFO-POD")
# modelcolors.append('#d62728')
modelnames.append("Spire 104 POD")
modelcolors.append(sp104podcolor)
modelnames.append("Spire 113 POD")
modelcolors.append(sp113podcolor)
modelnames.append("Spire 106 POD")
modelcolors.append(sp106podcolor)





df_leg = pd.DataFrame({"starts_colors": modelcolors})
fig.update_traces(showlegend=False).add_traces(
    [   go.Scattergl(name=modelnames[i],
               x=[pd.to_datetime( "181107-000000", format='%y%m%d-%H%M%S')],
               mode='lines',
               line = dict(shape = 'hv',  width=10),
               marker_color=c,
               showlegend=True)
        for i,c in enumerate((df_leg.loc[:,["starts_colors"]].values.ravel()))])
## Legend Control
fig.update_layout(legend=dict(
    yanchor="top",
    y=.78,
    xanchor="center",
    x=.45,
    orientation="h",
        font=dict(family='Arial',size=12,color='black'),
        bgcolor="white",
        bordercolor="darkgrey",
        borderwidth=0.5,)  )
################################################################################

fig.add_hline(y=0, line = dict(dash='solid', color = 'grey', width=1), row=3, col=1)

### UPDATE AXES 
fig.update_yaxes(title_text="Kp", 
                 exponentformat= 'power',
                 range=[0,7],
                 secondary_y=False,
                 row=1, col=1)
fig.update_yaxes(title_text="F10.7", 
                 exponentformat= 'power',
                 range=[35,170],
                 secondary_y=True,
                 tickfont=dict(color="blue"),
                 titlefont=dict(color="blue"),
                 row=1, col=1)
################################################################################

### SYLIZE LEGEND 
font_dict=dict(family='Arial',size=11,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = False
    else:
        if i < len(rownum):
            L_ticklabel = False
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
#                       tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)


# fig.update_xaxes(range=[pd.to_datetime( "181101-000000", format='%y%m%d-%H%M%S'),
#                         pd.to_datetime( "181130-000000", format='%y%m%d-%H%M%S')],row=1, col=1)

fig.update_xaxes(range=[pd.to_datetime('2022-06-01 20:54:00'), pd.to_datetime('2022-06-30 00:00:00')])

fig.update_yaxes(title_text="Orbit Averaged Density", 
                 type="log", 
                 exponentformat= 'power',row=2, col=1)
fig.update_yaxes(title_text="Mean Ratio of Orbit Avg.", 
                minor=dict(dtick=10, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
                 exponentformat= 'power',row=3, col=1)
# fig.update_yaxes(title_text="GFO_acc-ICE_pod %d", 
#                 minor=dict(dtick=10, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
#                  exponentformat= 'power',row=4, col=1)
# fig.update_yaxes(title_text="GFO_pod-ICE_pod %d", 
#                 minor=dict(dtick=10, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
#                  exponentformat= 'power',row=5, col=1)

fig.update_layout(margin=dict(l=20, r=20, t=30, b=20),)

fig.update_layout(title=f"Spire-FM104 Rho_x compared to GRACE-FO Acc., Norm 500km",
                  autosize=False,    width=1000,    height=800,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict, plot_bgcolor='white', 
                 )

fig.show(config=config)

# pio.write_image(fig, 'ICEvGFO.jpg', scale=5)


In [ ]:

# perc_stat['percdiff_sp104_log']

In [ ]:
signal.savgol_filter(perc_stat['percdiff_sp104_log'], 53, 3)

In [ ]:
perc_stat['percdiff_GFO_log']

In [ ]:
perc_stat['time_midpoint_sp104']

In [ ]:
perc_stat['time_midpoint_sp104']